In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2024-09-05 13:52:28--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: ‘datasource.zip’

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2024-09-05 13:52:32 (69.2 MB/s) - ‘datasource.zip’ saved [4249/4249]



In [11]:
import pandas as pd 
import glob 
import xml.etree.ElementTree as ET 
from datetime import datetime

In [2]:
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = car.find("year_of_manufacture").text
        price = car.find("price").text
        fuel = car.find("fuel").text
        dataframe = pd.concat([dataframe, pd.DataFrame({"car_model": [car_model], "year_of_manufacture": [year_of_manufacture], "price": [price], "fuel": [fuel]})], ignore_index=True)
    return dataframe

In [36]:
def extract(folder):
    extracted_data = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])

    for file in glob.glob(folder + "/*.csv"):
        extracted_data = pd.concat([extracted_data, extract_from_csv(file)], ignore_index=True)

    for file in glob.glob(folder + "/*.json"):
        extracted_data = pd.concat([extracted_data, extract_from_json(file)], ignore_index=True)

    for file in glob.glob(folder + "/*.xml"):
        extracted_data = pd.concat([extracted_data, extract_from_xml(file)], ignore_index=True)

    return extracted_data

In [26]:
def transform_data(data):
    # tranform price and round to upto 2 decimal places
    data["price"] = data["price"].transform(lambda x: round(float(x), 2))
    return data

In [27]:
def load(transform_data):
    transform_data.to_csv("transformed_data.csv", index=False)

In [28]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt", "a") as file:
        file.write(timestamp + ',' + message + '\n')

In [38]:
log("ETL Job Started")

log("Extract phase Started")
extracted_data = extract("datasource")
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform_data(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(transformed_data)

log("Load phase Ended")


/tmp/ipykernel_161/3061653740.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, extract_from_csv(file)], ignore_index=True)
